# Investment Comparison Tool

This notebook helps compare different investment types including:
- Stocks/ETFs
- Real Estate
- Bonds/Fixed Income

Use this to analyze returns, risks, and make informed investment decisions.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
from datetime import datetime, timedelta

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries imported successfully!")

## 1. Stocks/ETFs Analysis

In [ ]:
# Define stock/ETF investments
stocks = {
    'SPY': {'name': 'S&P 500 ETF', 'investment': 10000},
    'VTI': {'name': 'Total Stock Market ETF', 'investment': 10000},
    'QQQ': {'name': 'Nasdaq 100 ETF', 'investment': 10000}
}

# Fetch historical data
end_date = datetime.now()
start_date = end_date - timedelta(days=365*5)  # 5 years of data

stock_data = {}
for ticker in stocks.keys():
    stock_data[ticker] = yf.download(ticker, start=start_date, end=end_date, progress=False)

print("Stock data downloaded successfully!")

In [ ]:
# Calculate returns and metrics
def calculate_stock_metrics(ticker, data, initial_investment):
    returns = data['Adj Close'].pct_change().dropna()
    
    initial_price = data['Adj Close'].iloc[0]
    final_price = data['Adj Close'].iloc[-1]
    total_return = (final_price - initial_price) / initial_price
    final_value = initial_investment * (1 + total_return)
    
    annual_return = returns.mean() * 252  # Annualized
    annual_volatility = returns.std() * np.sqrt(252)  # Annualized
    sharpe_ratio = annual_return / annual_volatility if annual_volatility > 0 else 0
    
    return {
        'Total Return (%)': total_return * 100,
        'Annual Return (%)': annual_return * 100,
        'Annual Volatility (%)': annual_volatility * 100,
        'Sharpe Ratio': sharpe_ratio,
        'Initial Investment': initial_investment,
        'Final Value': final_value,
        'Gain/Loss': final_value - initial_investment
    }

stock_metrics = {}
for ticker, info in stocks.items():
    stock_metrics[info['name']] = calculate_stock_metrics(ticker, stock_data[ticker], info['investment'])

stock_df = pd.DataFrame(stock_metrics).T
print("\nStock/ETF Performance Metrics:")
stock_df

In [ ]:
# Plot stock performance
plt.figure(figsize=(14, 6))

for ticker in stocks.keys():
    normalized = stock_data[ticker]['Adj Close'] / stock_data[ticker]['Adj Close'].iloc[0]
    plt.plot(normalized, label=stocks[ticker]['name'])

plt.title('Normalized Stock/ETF Performance', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Normalized Price (Base = 1.0)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Real Estate Investment Analysis

In [ ]:
# Define real estate investments
real_estate_investments = [
    {
        'name': 'Rental Property A',
        'purchase_price': 250000,
        'down_payment': 50000,
        'monthly_rent': 2000,
        'monthly_expenses': 600,  # mortgage, taxes, insurance, maintenance
        'appreciation_rate': 0.03,  # 3% annual
        'years': 5
    },
    {
        'name': 'REIT Investment',
        'purchase_price': 50000,
        'down_payment': 50000,
        'monthly_rent': 200,  # dividend income
        'monthly_expenses': 0,
        'appreciation_rate': 0.05,  # 5% annual
        'years': 5
    }
]

def calculate_real_estate_metrics(investment):
    monthly_cash_flow = investment['monthly_rent'] - investment['monthly_expenses']
    annual_cash_flow = monthly_cash_flow * 12
    total_cash_flow = annual_cash_flow * investment['years']
    
    future_value = investment['purchase_price'] * (1 + investment['appreciation_rate']) ** investment['years']
    equity_gain = future_value - investment['purchase_price']
    
    total_return = total_cash_flow + equity_gain
    roi = (total_return / investment['down_payment']) * 100
    annual_roi = roi / investment['years']
    
    return {
        'Initial Investment': investment['down_payment'],
        'Monthly Cash Flow': monthly_cash_flow,
        'Total Cash Flow': total_cash_flow,
        'Property Appreciation': equity_gain,
        'Total Return': total_return,
        'ROI (%)': roi,
        'Annual ROI (%)': annual_roi,
        'Final Value': investment['down_payment'] + total_return
    }

real_estate_metrics = {inv['name']: calculate_real_estate_metrics(inv) for inv in real_estate_investments}
real_estate_df = pd.DataFrame(real_estate_metrics).T

print("\nReal Estate Investment Metrics:")
real_estate_df

## 3. Bonds/Fixed Income Analysis

In [ ]:
# Define bond investments
bonds = [
    {
        'name': '10-Year Treasury',
        'principal': 50000,
        'annual_rate': 0.04,  # 4% yield
        'years': 5,
        'compounding': 'semi-annual'
    },
    {
        'name': 'Corporate Bond',
        'principal': 50000,
        'annual_rate': 0.055,  # 5.5% yield
        'years': 5,
        'compounding': 'semi-annual'
    },
    {
        'name': 'CD (Certificate of Deposit)',
        'principal': 50000,
        'annual_rate': 0.045,  # 4.5% APY
        'years': 5,
        'compounding': 'monthly'
    }
]

def calculate_bond_metrics(bond):
    # Determine compounding frequency
    if bond['compounding'] == 'annual':
        n = 1
    elif bond['compounding'] == 'semi-annual':
        n = 2
    elif bond['compounding'] == 'quarterly':
        n = 4
    elif bond['compounding'] == 'monthly':
        n = 12
    else:
        n = 1
    
    # Calculate future value with compound interest
    future_value = bond['principal'] * (1 + bond['annual_rate'] / n) ** (n * bond['years'])
    total_interest = future_value - bond['principal']
    total_return_pct = (total_interest / bond['principal']) * 100
    annual_return = total_return_pct / bond['years']
    
    return {
        'Initial Investment': bond['principal'],
        'Annual Rate (%)': bond['annual_rate'] * 100,
        'Years': bond['years'],
        'Total Interest': total_interest,
        'Final Value': future_value,
        'Total Return (%)': total_return_pct,
        'Annual Return (%)': annual_return
    }

bond_metrics = {bond['name']: calculate_bond_metrics(bond) for bond in bonds}
bond_df = pd.DataFrame(bond_metrics).T

print("\nBonds/Fixed Income Metrics:")
bond_df

## 4. Comprehensive Comparison

In [ ]:
# Combine all investments for comparison
comparison_data = []

# Add stocks
for name, metrics in stock_metrics.items():
    comparison_data.append({
        'Investment': name,
        'Type': 'Stock/ETF',
        'Initial Investment': metrics['Initial Investment'],
        'Final Value': metrics['Final Value'],
        'Gain/Loss': metrics['Gain/Loss'],
        'Return (%)': metrics['Total Return (%)'],
        'Risk (Volatility %)': metrics['Annual Volatility (%)']
    })

# Add real estate
for name, metrics in real_estate_metrics.items():
    comparison_data.append({
        'Investment': name,
        'Type': 'Real Estate',
        'Initial Investment': metrics['Initial Investment'],
        'Final Value': metrics['Final Value'],
        'Gain/Loss': metrics['Total Return'],
        'Return (%)': metrics['ROI (%)'],
        'Risk (Volatility %)': 10  # Estimated for real estate
    })

# Add bonds
for name, metrics in bond_metrics.items():
    comparison_data.append({
        'Investment': name,
        'Type': 'Bond/Fixed Income',
        'Initial Investment': metrics['Initial Investment'],
        'Final Value': metrics['Final Value'],
        'Gain/Loss': metrics['Total Interest'],
        'Return (%)': metrics['Total Return (%)'],
        'Risk (Volatility %)': 2  # Low risk for bonds
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Return (%)', ascending=False)

print("\nComprehensive Investment Comparison:")
comparison_df

In [ ]:
# Visualization: Returns Comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart of returns
colors = {'Stock/ETF': '#3498db', 'Real Estate': '#2ecc71', 'Bond/Fixed Income': '#e74c3c'}
comparison_df['Color'] = comparison_df['Type'].map(colors)

axes[0].barh(comparison_df['Investment'], comparison_df['Return (%)'], color=comparison_df['Color'])
axes[0].set_xlabel('Return (%)', fontsize=12)
axes[0].set_title('Investment Returns Comparison', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Risk vs Return scatter plot
for inv_type, color in colors.items():
    subset = comparison_df[comparison_df['Type'] == inv_type]
    axes[1].scatter(subset['Risk (Volatility %)'], subset['Return (%)'], 
                   c=color, label=inv_type, s=200, alpha=0.7, edgecolors='black')

axes[1].set_xlabel('Risk (Volatility %)', fontsize=12)
axes[1].set_ylabel('Return (%)', fontsize=12)
axes[1].set_title('Risk vs Return Profile', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Summary statistics
print("\n=== Investment Summary ===")
print(f"\nTotal Invested: ${comparison_df['Initial Investment'].sum():,.2f}")
print(f"Total Final Value: ${comparison_df['Final Value'].sum():,.2f}")
print(f"Total Gain/Loss: ${comparison_df['Gain/Loss'].sum():,.2f}")
print(f"\nBest Performing Investment: {comparison_df.iloc[0]['Investment']}")
print(f"Return: {comparison_df.iloc[0]['Return (%)']:.2f}%")

print("\n=== Risk-Adjusted Returns (Sharpe-like Ratio) ===")
comparison_df['Risk-Adjusted Return'] = comparison_df['Return (%)'] / comparison_df['Risk (Volatility %)']
best_risk_adjusted = comparison_df.sort_values('Risk-Adjusted Return', ascending=False).iloc[0]
print(f"Best Risk-Adjusted Investment: {best_risk_adjusted['Investment']}")
print(f"Risk-Adjusted Score: {best_risk_adjusted['Risk-Adjusted Return']:.2f}")

## 5. Custom Investment Analysis

Use the cells below to add your own investments and analyze them.

In [ ]:
# Add your custom investment here
# Example:
# custom_investment = {
#     'name': 'My Investment',
#     'initial': 10000,
#     'final': 12000,
#     'years': 3
# }
